In [5]:
# https://developers.line.me/console/channel/1578247603/basic/
from flask import Flask, request, abort
import re
from linebot import (
    LineBotApi, WebhookHandler
)
from linebot.exceptions import (
    InvalidSignatureError
)
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage,TemplateSendMessage,ConfirmTemplate,PostbackTemplateAction, MessageTemplateAction, URITemplateAction,
    CarouselColumn,ButtonsTemplate,CarouselTemplate,ImageSendMessage
)
import requests
import numpy as np
from bs4 import BeautifulSoup
import random
from dateutil.parser import parse
import json
import pandas as pd
from lxml import etree
from selenium import webdriver
from imgurpython import ImgurClient
import time

app = Flask(__name__)

line_bot_api = LineBotApi('yq5CKa/4t0cdSwbwyh65MUgiHCcfmrOOabQoFYwzCwmgLHeNzjhJHQXE4Iv3CYfZq9OQm5uRSK/2tAm4LNwUO22hONQCfvq61JRO7wRzgVBkmbmdbk/a628gRF7+q2mhoYtn+V9jL2ZFxPWyoimPuQdB04t89/1O/w1cDnyilFU=')
handler = WebhookHandler('65466c485d06fc4e4b929ebf252d8102')

def mlb():
    res = requests.get('https://www.lottonavi.com/scoreboard/mlb/')
    res.encoding = 'utf-8'
    soup = BeautifulSoup(res.text)
    mlb = pd.read_html(res.text,attrs={'class':'table table-bordered scbf'})
    content = ""
    for i in range(14):
        try:
            split = '-------------------------'
            try:
                if np.isnan(mlb[i][11].iloc[1]) == True:
                    title = '比賽即將開始:',mlb[i][0].iloc [0]
                    team_1 = '隊伍:',mlb[i][0].iloc [1],0
                    team_2 = '隊伍:',mlb[i][0].iloc [2],0
                    content += '{}\n{}\n{}\n{}\n'.format(title,split, team_1, team_2)
                else:
                    team_1 = '隊伍:'+mlb[i][0].iloc[1],int(mlb[i][11].iloc[1])
                    team_2 = '隊伍:'+mlb[i][0].iloc[2],int(mlb[i][11].iloc[2])
                    content += '{}\n{}\n{}\n'.format(split, team_1, team_2)
            except:
                team_1 = '隊伍:'+mlb[i][0].iloc[1],int(mlb[i][11].iloc[1])
                team_2 = '隊伍:'+mlb[i][0].iloc[2],int(mlb[i][11].iloc[2])
                content += '{}\n{}\n{}\n'.format(split, team_1, team_2)
        except:
            content = '今日無賽事'
    return content

def mlbrank():
    req = requests.get('http://www.azonnet.com/sports3/?page=mlb/standings')
    df = pd.read_html(req.text,attrs={'class':'table table-condensed table-striped valign-mid-all center-label-all left-label-col-2'},encoding='BIG5')[0]
    for i in range(1,6):  
        df1 = pd.read_html(req.text,attrs={'class':'table table-condensed table-striped valign-mid-all center-label-all left-label-col-2'},encoding='BIG5')[i]
        df = pd.concat([df,df1])
    df = df.reset_index(drop=True)
    content = ''
    for i in range(len(df)):
        team = str(df[1][i]) + str(df[5][i])
        diff = ' '
        content += '{}\n{}\n'.format(team, diff)
    return content

def npb():
    res = requests.get('https://www.lottonavi.com/scoreboard/npb/')
    res.encoding = 'utf-8'
    soup = BeautifulSoup(res.text)
    try:
        npb = pd.read_html(res.text,attrs={'class':'table table-bordered scbf'})
        content = ""
        for i in range(7):
            try:
                split = '-----------------------'
                try:
                    if np.isnan(npb[i][11].iloc[1]) == True:
                        title = '比賽即將開始:',npb[i][0].iloc [0]
                        team_1 = '隊伍:',npb[i][0].iloc [1],0
                        team_2 = '隊伍:',npb[i][0].iloc [2],0
                        content += '{}\n{}\n{}\n{}\n'.format(split, title, team_1, team_2)
                    else:
                        team_1 = '隊伍:'+npb[i][0].iloc [1],int(npb[i][11].iloc[1])
                        team_2 = '隊伍:'+npb[i][0].iloc[2],int(npb[i][11].iloc[2])
                        content += '{}\n{}\n{}\n'.format(split, team_1, team_2)
                except:
                    team_1 = '隊伍:'+npb[i][0].iloc [1],int(npb[i][11].iloc[1])
                    team_2 = '隊伍:'+npb[i][0].iloc[2],int(npb[i][11].iloc[2])
                    content += '{}\n{}\n{}\n'.format(split, team_1, team_2)
            except:
                pass
    except:
        content = '今日無賽事'
    return content

def npbrank():
    content=''
    a = ['--------中央聯盟--------','廣島','養樂多','巨人','阪神','橫濱','中日','--------太平洋聯盟--------','西武','日本火腿','軟銀','歐力士','羅德','樂天']
    b = [' ',39,32,32,31,30,31,' ',39,40,36,35,35,25]
    c = [' ',28,34,36,35,35,38,' ',28,30,32,32,33,44]
    for i,j,k in list(zip(a,b,c)):
        team = str(i),str(j)+'勝',str(k)+'敗'
        content += "{}\n".format(team)
    return content

def cpbl():
    content = '今日無賽事'
    return content

def cpblrank():
    content=''
    a = ['Lamigo','統一7-ELEVEn','富邦','中信兄弟']
    b = [38,35,27,20]
    c = [22,25,33,30]
    for i,j,k in list(zip(a,b,c)):
        team = str(i),str(j)+'勝',str(k)+'敗'
        content += "{}\n".format(team)
    return content

def soccer():  
    r=requests.get('https://tw.sports.yahoo.com/soccer/world-cup/scoreboard/')
    re = r.content
    soup=BeautifulSoup(re,"html.parser")
    f=soup.select('.Py\(10px\)')
    content='今日比賽\n'
    for a in f:
        title2=a.select('div')[0].text
        content += "{}\n".format(title2)
    content = content.replace('結束','結束~~')
    return content

def soccer_rank():  
    stan = requests.get('https://tw.sports.yahoo.com/soccer/world-cup/standings/')
    df = pd.read_html(stan.text,attrs={'class':'W(100%)'})
    content = ''
    a = [0,1,2,3,4,5,6,7]
    b = ['A','B','C','D','E','F','G','H']
    for i,j in list(zip(a,b)):
        title = '分區:'+str(j)+'組'
        rank = str(df[i][str(j)+' 組'].values)
        content += '{}\n{}\n'.format(title,rank)
    return content

def nba():
    content = '今日無賽事'
    return content

def nbarank():
    url = 'https://www.msn.com/zh-tw/sports/nba/standings/sp-vw-con'
    df = pd.read_html(requests.get(url).text, attrs={'class': 'tableview standing'})
    # 東區
    east = df[0]
    columns = east.columns
    east = east.drop("隊伍",axis=1)
    east = east.drop("負",axis=1)
    east.columns = columns[:-2]
    # 西區
    west = df[1]
    columns = west.columns
    west = west.drop("隊伍",axis=1)
    west = west.drop("負",axis=1)
    west.columns = columns[:-2]
    content  ='-----------\n東區 \n-----------\n'
    for i,j,k in list(zip(east['隊伍'],east['勝'],east['負'])):
        team_e = str(i.split('-')[0]),str(j)+'勝',str(k)+'敗'
        content += "{}\n".format(team_e)
    content +='-----------\n西區 \n-----------\n'
    for i,j,k in list(zip(west['隊伍'],west['勝'],west['負'])):
        team_w = str(i.split('-')[0]),str(j)+'勝',str(k)+'敗'
        content += "{}\n".format(team_w)
    return content

def sbl():
    content = '今日無賽事'
    return content

def sblrank():
    content=''
    a = ['裕隆納智捷','桃園璞園建築','金門酒廠','臺北達欣工程','富邦勇士','台灣啤酒','臺灣銀行']
    b = [19,18,16,15,15,12,10]
    c = [11,12,14,15,15,18,20]
    for i,j,k in list(zip(a,b,c)):
        team = str(i),str(j)+'勝',str(k)+'敗'
        content += "{}\n".format(team)
    return content


def tennis():
    driver = webdriver.Chrome(executable_path='/Users/andy/anaconda/selenium/chromedriver')
    driver.get('http://tennis.7m.com.cn')
    page = etree.HTML(driver.page_source)
    driver.close()
    ids=[]
    ids1=[]
    content = ''
    for i in page.xpath('//tbody/tr[@class="wqbg1"]'):
        if not i in ids:
            ids.append(i.attrib['id'].split('_')[1])
    [ids1.append(i) for i in ids if i not in ids1]
    ids=[]
    ids2=[]
    for i in page.xpath('//tbody/tr[@class="wqbg2"]'):

        if not i in ids:
            ids.append(i.attrib['id'].split('_')[1])

    [ids2.append(i) for i in ids if i not in ids2]  
    a=[]
    b=[]
    b1=[]
    for i in ids1:
        for num in range(1,11):
            for ele1 in page.xpath('//*[@id='+'"'+'r1_'+i+'"'+']/td['+str(num)+']'):
                a.append(ele1.text)
    for i,j in zip(range(0,len(a),10),range(10,len(a),10)):
        b.append(' '.join(a[i:j]))
    c=[]
    d=[]
    d1=[]
    for i in ids1:
        for num in range(1,11):
            for ele2 in page.xpath('//*[@id='+'"'+'r2_'+i+'"'+']/td['+str(num)+']'):
                c.append(ele2.text)
    for i,j in zip(range(0,len(c),7),range(7,len(c),7)):
        d.append(c[i:j])
        d1.append(' '.join(c[i:j]))
    e=[]
    f=[]
    for i in ids2:
        for num in range(1,11):
            for ele1 in page.xpath('//*[@id='+'"'+'r1_'+i+'"'+']/td['+str(num)+']'):
                e.append(ele1.text)
    for i,j in zip(range(0,len(a),10),range(10,len(a),10)):
        f.append(' '.join(e[i:j]))
    g=[]
    h=[]
    h1=[]
    for i in ids2:
        for num in range(1,11):
            for ele2 in page.xpath('//*[@id='+'"'+'r2_'+i+'"'+']/td['+str(num)+']'):
                g.append(ele2.text)
    for i,j in zip(range(0,len(c),7),range(7,len(c),7)):
        h.append(g[i:j])
        h1.append(' '.join(g[i:j]))
    content =''
    for i in range(len(b)):
        split = '-------------------------'
        name_1 = b[i].split(' ')[0]
        player_1 = b[i].split(' ')[3]+' VS '+d1[i].split(' ')[0]
        score_1 = b[i].split(' ')[9]+' : '+d1[i].split(' ')[6]
        name_2 = f[i].split(' ')[0]
        player_2 = f[i].split(' ')[3]+' VS '+h1[i].split(' ')[0]
        score_2 = f[i].split(' ')[9]+' : '+h1[i].split(' ')[6]
        content += '{}\n{}\n{}\n{}\n{}\n{}\n{}\n{}\n'.format(split, name_1, player_1, score_1, split, name_2, player_2, score_2)
    return content

def tennis_atprank():
    req = requests.get('https://www.sportsv.net/tennis/rankings')
    df1 = pd.DataFrame()
    df = pd.read_html(req.text,attrs={'class':'tennis-rank'})[0]
    df1['排名'] = df[' 單打排名']['排名']
    df1['球員'] = df['Unnamed: 1_level_0']['球員']
    df1['積分'] = df['Unnamed: 2_level_0']['積分']
    content =''
    for i in range(0,10):
        split = '-------------------------'
        rank = df1['排名'][i]
        player = df1['球員'][i]
        score = '積分:'+str(df1['積分'][i])
        content += '{}\n{}\n{}\n{}\n'.format(split,rank,player,score)
    return content

def tennis_wtarank():
    req = requests.get('https://www.sportsv.net/tennis/rankings/WTA_top100')
    df1 = pd.DataFrame()
    df = pd.read_html(req.text,attrs={'class':'tennis-rank'})[0]
    df1['排名'] = df[' 單打排名']['排名']
    df1['球員'] = df['Unnamed: 1_level_0']['球員']
    df1['積分'] = df['Unnamed: 2_level_0']['積分']
    content =''
    for i in range(0,10):
        split = '-------------------------'
        rank = df1['排名'][i]
        player = df1['球員'][i]
        score = '積分:'+str(df1['積分'][i])
        content += '{}\n{}\n{}\n{}\n'.format(split,rank,player,score)
    return content

def mlb_lottery():
    driver = webdriver.Chrome('/Users/andy/anaconda/selenium/chromedriver')
    driver.get("https://m.sportslottery.com.tw/zh/web/guest/sports-betting#m/all")
    driver.find_element_by_id("topSport_s-443").click()
    time.sleep(3)
    soup = BeautifulSoup(driver.page_source)
    table = soup.find(id = "topEventsContainer")
    content = ''
    for cells in table:
        for i in (cells.select(".bets.live.clearfix")):
            split = '-------------------------'
            content += '{}\n{}\n'.format(split,i.text[4:])
    driver.close()
    return content

def soccer_lottery():
    driver = webdriver.Chrome('/Users/andy/anaconda/selenium/chromedriver')
    driver.get("https://m.sportslottery.com.tw/zh/web/guest/sports-betting#m/all")
    driver.find_element_by_id("topSport_s-441").click()
    time.sleep(3)
    soup = BeautifulSoup(driver.page_source)
    table = soup.find(id = "topEventsContainer")
    content1 = ''
    for cells in table:
        for content in (cells.select(".bets.live.clearfix")):
            split = '-------------------------'
            content1 += '{}\n{}\n'.format(split,content.text[4:])
    driver.close()
    return content1

def tennis_lottery():
    driver = webdriver.Chrome('/Users/andy/anaconda/selenium/chromedriver')
    driver.get("https://m.sportslottery.com.tw/zh/web/guest/sports-betting#m/all")
    driver.find_element_by_id("topSport_s-445").click()
    time.sleep(3)
    soup = BeautifulSoup(driver.page_source)
    table = soup.find(id = "topEventsContainer")
    content1 = ''
    for cells in table:
        for content in (cells.select(".bets.live.clearfix")):
            split = '-------------------------'
            content1 += '{}\n{}\n'.format(split,content.text[4:])
    driver.close()
    return content1

@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    line=event.message.text
    
    if line == "MLB比分":
        content = mlb()
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text=content))
        return 0
    
    if line == "日職比分":
        content = npb()
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text=content))
        return 0
    
    if line == "中職比分":
        content = cpbl()
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text=content))
        return 0
    
    if line == "世足比分":
        content = soccer()
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text=content))
        return 0
    
    if line == "網球比分":
        content = tennis()
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text=content))
        return 0
    
    if line == "NBA比分":
        content = nba()
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text=content))
        return 0
    
    if line == "SBL比分":
        content = sbl()
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text=content))
        return 0
    
    if line == "MLB運彩":
        content = mlb_lottery()
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text=content))
        return 0
    
    if line == "世足運彩":
        content = soccer_lottery()
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text=content))
        return 0
    
    if line == "網球運彩":
        content = tennis_lottery()
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text=content))
        return 0
    
    if line == "MLB戰績":
        content = mlbrank()
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text=content))
        return 0
    
    if line == "日職戰績":
        content = npbrank()
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text=content))
        return 0
    
    if line == "中職戰績":
        content = cpblrank()
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text=content))
        return 0
    
    if line == "NBA戰績":
        content = nbarank()
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text=content))
        return 0
    
    if line == "SBL戰績":
        content = sblrank()
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text=content))
        return 0
    
    if line == "男子單打排名":
        content = tennis_atprank()
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text=content))
        return 0
    
    if line == "女子單打排名":
        content = tennis_wtarank()
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text=content))
        return 0
    
    if line == "世足分組排名":
        content = soccer_rank()
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text=content))
        return 0

    ###button template###
    if line=='運彩':
        buttons_template_message = TemplateSendMessage(
        alt_text='Buttons template',
        template=ButtonsTemplate(
            thumbnail_image_url='https://metrouk2.files.wordpress.com/2017/08/aw-lottery-chances-of-winning.jpg?w=748&h=427&crop=1',
            title='運彩賠率',
            text='Please select',
            actions=[
                MessageTemplateAction(
                    label='美國職棒',
                    text='MLB運彩'
                ),
                MessageTemplateAction(
                    label='世足杯',
                    text='世足運彩'
                ),
                MessageTemplateAction(
                    label='網球',
                    text='網球運彩'
                )
            ]
        )
    )
        line_bot_api.reply_message(event.reply_token, buttons_template_message)
        return 0
    
    ###button template###
    if line=='新聞':
        buttons_template_message = TemplateSendMessage(
        alt_text='Buttons template',
        template=ButtonsTemplate(
            thumbnail_image_url='https://st3.depositphotos.com/1007566/12975/v/950/depositphotos_129757162-stock-illustration-reporter-breaking-news-icon.jpg',
            title='各大體育新聞',
            text='Please select',
            actions=[
                URITemplateAction(
                    label='MSN運動',
                    uri = 'https://www.msn.com/zh-tw/sports'
                ),
                URITemplateAction(
                    label='運動視界',
                    uri = 'https://www.sportsv.net'
                ),
                URITemplateAction(
                    label='聯合新聞網',
                    uri = 'https://udn.com/news/cate/2/7227'
                ),
                URITemplateAction(
                    label='Yahoo奇摩運動',
                    uri = 'https://tw.sports.yahoo.com'
                )
            ]
        )
    )
        line_bot_api.reply_message(event.reply_token, buttons_template_message)
        return 0
    
    if line=='美國職棒':
        buttons_template_message = TemplateSendMessage(
        alt_text='Buttons template',
        template=ButtonsTemplate(
            thumbnail_image_url='https://static.techspot.com/images2/news/bigimage/2018/03/2018-03-09-image-14.jpg',
            title='美國職棒 MLB',
            text='Please select',
            actions=[
                MessageTemplateAction(
                    label='今日戰況',
                    text='MLB比分'
                ),
                MessageTemplateAction(
                    label='戰績排名',
                    text='MLB戰績'
                ),
                URITemplateAction(
                    label='官方網站',
                    uri = 'https://www.mlb.com'
                )
            ]
        )
    )
        line_bot_api.reply_message(event.reply_token, buttons_template_message)
        return 0
    
    if line=='日本職棒':
        buttons_template_message = TemplateSendMessage(
        alt_text='Buttons template',
        template=ButtonsTemplate(
            thumbnail_image_url='https://upload.wikimedia.org/wikipedia/commons/thumb/7/7f/NPB_logo.svg/1176px-NPB_logo.svg.png',
            title='日本職棒 NPB',
            text='Please select',
            actions=[
                MessageTemplateAction(
                    label='今日戰況',
                    text='日職比分'
                ),
                MessageTemplateAction(
                    label='戰績排名',
                    text='日職戰績'
                ),
                URITemplateAction(
                    label='官方網站',
                    uri = 'http://npb.jp'
                )
            ]
        )
    )
        line_bot_api.reply_message(event.reply_token, buttons_template_message)
        return 0
    
    if line=='中華職棒':
        buttons_template_message = TemplateSendMessage(
        alt_text='Buttons template',
        template=ButtonsTemplate(
            thumbnail_image_url='https://upload.wikimedia.org/wikipedia/zh/thumb/7/74/CPBL_logo.svg/1200px-CPBL_logo.svg.png',
            title='中華職棒 CPBL',
            text='Please select',
            actions=[
                MessageTemplateAction(
                    label='今日戰況',
                    text='中職比分'
                ),
                MessageTemplateAction(
                    label='戰績排名',
                    text='中職戰績'
                ),
                URITemplateAction(
                    label='官方網站',
                    uri = 'http://www.cpbl.com.tw'
                )
            ]
        )
    )
        line_bot_api.reply_message(event.reply_token, buttons_template_message)
        return 0
    
    if line=='NBA':
        buttons_template_message = TemplateSendMessage(
        alt_text='Buttons template',
        template=ButtonsTemplate(
            thumbnail_image_url='https://vignette.wikia.nocookie.net/nba-family/images/0/02/Nba-logo.jpg/revision/latest?cb=20151119043138',
            title='美國職籃 NBA',
            text='Please select',
            actions=[
                MessageTemplateAction(
                    label='今日戰況',
                    text='NBA比分'
                ),
                MessageTemplateAction(
                    label='戰績排名',
                    text='NBA戰績'
                ),
                URITemplateAction(
                    label='官方網站',
                    uri = 'http://www.nba.com/?50#/'
                )
            ]
        )
    )
        line_bot_api.reply_message(event.reply_token, buttons_template_message)
        return 0
    
    if line=='台灣職籃':
        buttons_template_message = TemplateSendMessage(
        alt_text='Buttons template',
        template=ButtonsTemplate(
            thumbnail_image_url='https://4.bp.blogspot.com/-3q4lNJ_Lglw/WNi3lVAWi3I/AAAAAAAAELo/b3nGB0KycdwwAfw6mAhhG29buen5WG6ZACLcB/s1600/SBL_logo-95c465970fd6ab7b38d1b1beff2ff8ccd2bd64fbd1f7f471f3b4dc5bd7da2a0e.png',
            title='台灣職籃 SBL',
            text='Please select',
            actions=[
                MessageTemplateAction(
                    label='今日戰況',
                    text='SBL比分'
                ),
                MessageTemplateAction(
                    label='戰績排名',
                    text='SBL戰績'
                ),
                URITemplateAction(
                    label='官方網站',
                    uri = 'http://sbl.choxue.com'
                )
            ]
        )
    )
        line_bot_api.reply_message(event.reply_token, buttons_template_message)
        return 0
    
    if line=='網球':
        buttons_template_message = TemplateSendMessage(
        alt_text='Buttons template',
        template=ButtonsTemplate(
            thumbnail_image_url='https://www.tennisworldusa.org/imgb/61221/roger-federer-explains-why-younger-players-are-not-winning-grand-slams.jpg',
            title='網球',
            text='Please select',
            actions=[
                MessageTemplateAction(
                    label='今日戰況',
                    text='網球比分'
                ),
                MessageTemplateAction(
                    label='男子單打排名',
                    text='男子單打排名'
                ),
                MessageTemplateAction(
                    label='女子單打排名',
                    text='女子單打排名'
                )
            ]
        )
    )
        line_bot_api.reply_message(event.reply_token, buttons_template_message)
        return 0
    
    if line=='世足盃':
        buttons_template_message = TemplateSendMessage(
        alt_text='Buttons template',
        template=ButtonsTemplate(
            thumbnail_image_url='https://www.udesa.edu.ar/sites/default/files/logotipo_rusia_2018.jpeg',
            title='世界盃足球賽',
            text='Please select',
            actions=[
                MessageTemplateAction(
                    label='今日戰況',
                    text='世足比分'
                ),
                MessageTemplateAction(
                    label='分組排名',
                    text='世足分組排名'
                ),
                URITemplateAction(
                    label='官方網站',
                    uri = 'https://www.fifa.com/worldcup/'
                )
            ]
        )
    )
        line_bot_api.reply_message(event.reply_token, buttons_template_message)
        return 0
    
    ######
    if line=='選單':
        carousel_template_message = TemplateSendMessage(
        alt_text='Carousel template',
        template=CarouselTemplate(
            columns=[
                CarouselColumn(
                    thumbnail_image_url='https://cdn-images-1.medium.com/max/1600/1*RdELyL9PcqV6NM-wP0Zt3A.jpeg',
                    title='棒球',
                    text=' ',
                    actions=[
                        PostbackTemplateAction(
                            label='美國職棒 MLB',
                            text='美國職棒',
                            data='action=buy&itemid=2'
                        ),
                        MessageTemplateAction(
                            label='日本職棒 NPB',
                            text='日本職棒'
                        ),
                        MessageTemplateAction(
                            label='中華職棒 CPBL',
                            text='中華職棒'
                        ),
                    ]
                ),
                CarouselColumn(
                    thumbnail_image_url='https://usathoopshype.files.wordpress.com/2017/11/lebron-durant-finals.jpg?w=1000&h=600&crop=1',
                    title='籃球',
                    text=' ',
                    actions=[
                        PostbackTemplateAction(
                            label='NBA',
                            text='NBA',
                            data='action=buy&itemid=1'
                        ),
                        MessageTemplateAction(
                            label='台灣職籃',
                            text='台灣職籃'
                        ),
                        MessageTemplateAction(
                            label=' ',
                            text=' '
                        )
                    ]
                ),
                CarouselColumn(
                    thumbnail_image_url='https://www.tennisworldusa.org/imgb/61221/roger-federer-explains-why-younger-players-are-not-winning-grand-slams.jpg',
                    title='網球',
                    text=' ',
                    actions=[
                        PostbackTemplateAction(
                            label='ATP & WTA',
                            text='網球',
                            data='action=buy&itemid=2'
                        ),
                        MessageTemplateAction(
                            label=' ',
                            text=' '
                        ),
                        MessageTemplateAction(
                            label=' ',
                            text=' '
                        )
                    ]
                ),
                CarouselColumn(
                    thumbnail_image_url='https://s314.siliconimg.com/kb/content_images/2016/05/21/1090716/1463810076_460.jpg?imageMogr2/strip/interlace/0/quality/85/format/jpg',
                    title='足球',
                    text=' ',
                    actions=[
                        PostbackTemplateAction(
                            label='世足',
                            text='世足盃',
                            data='action=buy&itemid=1'
                        ),
                        MessageTemplateAction(
                            label=' ',
                            text=' '
                        ),
                        MessageTemplateAction(
                            label=' ',
                            text=' '
                        )
                    ]
                )
            ]
        )
    )
        line_bot_api.reply_message(event.reply_token, carousel_template_message)
        return 0
    ####
    else:
        line_bot_api.reply_message(event.reply_token,TextSendMessage(text=event.message.text))


if __name__ == "__main__":
    app.run()

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [28/Jun/2018 15:14:29] "POST /callback HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2018 15:27:17] "POST /callback HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2018 15:27:21] "POST /callback HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2018 15:27:24] "POST /callback HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2018 15:27:25] "POST /callback HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2018 15:27:33] "POST /callback HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2018 15:27:34] "POST /callback HTTP/1.1" 200 -
/Users/andy/anaconda/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/andy/anaconda/lib/python3.6/runp